# Train HeterSUMGraphe

In [32]:
import pandas as pd
import argparse
from time import time
import torch
import numpy as np
import random
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from sklearn.utils import shuffle
import json
import os

from tqdm import tqdm

from utils.create_graph_dataset import create_graph_dataset
from utils.GloveMgr import GloveMgr

In [2]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

## (Hyper-)parameters

In [3]:
# Parse args if script mode
parser = argparse.ArgumentParser(description='extractive summary')

parser.add_argument('-is_graphic',type=int,default=1,choices=[0,1])
parser.add_argument('-gpu_num',type=int,default=0)
parser.add_argument('-batch_size',type=int,default=32)
parser.add_argument('-epochs',type=int,default=100)
parser.add_argument('-dataset',type=str,default="data/nyt_corpus_LDC2008T19_50.json")

args = None

if is_notebook():
    args = parser.parse_args("")
else:
    args = parser.parse_args()

In [4]:
# parameters
is_graphic = args.is_graphic != 0
cuda_num = args.gpu_num
embed_name = "glove.6B.300"

# hyper-parameters
vocab_size = 50000
batch_size = args.batch_size
epochs = args.epochs
learning_rate = 5e-4
early_stopping = 3
model_name = "HeterSUMGraph"
sub_folder_name = "model_name__{}__time__{}__embed_name__{}__lr__{}__batch_size__{}__vocab_size__{}__cuda_num__{}__early_stopping__{}".format(model_name, time(), embed_name, learning_rate, batch_size, vocab_size, cuda_num, early_stopping)
checkpoints_folder = "./checkpoints/" + sub_folder_name
average_proportion_of_sentences_per_document = 0.2670278281534701
average_number_of_sentences_per_document = 6.061850780738518

# print
print("parse:")
print("is_graphic:", is_graphic)
print("cuda_num:", cuda_num)
print("epochs", epochs)
print("batch_size", batch_size)

parse:
is_graphic: True
cuda_num: 0
epochs 100
batch_size 32


## PyTorch initialisation

In [5]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [6]:
# Check if a GPU is available
if torch.cuda.is_available():
    # Display the number of available GPUs
    print(f"Number of available GPUs: {torch.cuda.device_count()}")
    # Display the name of each GPU
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU available.")

Number of available GPUs: 1
GPU 0: NVIDIA GeForce RTX 3060


In [7]:
if torch.cuda.is_available():  
  dev = "cuda:" + str(cuda_num) 
else:  
  dev = "cpu" 

device = torch.device(dev)
device

device(type='cuda', index=0)

## load data

In [8]:
df = pd.read_json(args.dataset)
df = shuffle(df)

df_test = df.iloc[0:3452]
df_val = df.iloc[3452:7452]
df_train = df.iloc[7452:8000]

In [9]:
glovemgr = GloveMgr("./data/glove.6B/glove.6B.300d.txt", vocab_size=vocab_size)

In [10]:
tfidfs_sent = pd.read_json("data/nyt50_sent_tfidf.json")
tfidfs_sent = tfidfs_sent["tfidf"].to_numpy()

In [11]:
tfidfs_dataset = json.load(open("./data/nyt50_dataset_tfidf.json"))
tfidfs_dataset_sorted = sorted(tfidfs_dataset, key=tfidfs_dataset.get)
word_blacklist = set(tfidfs_dataset_sorted[:int(0.1*len(tfidfs_dataset_sorted))])

In [50]:
train_dataset = create_graph_dataset(df=df_train, tfidfs_sent=tfidfs_sent, glovemgr=glovemgr, word_blacklist = word_blacklist, doc_column_name="docs", labels_column_name="labels", is_sep_n=False, remove_stop_word = False, stemming=False, trunc_sent=-1, padding_sent=-1, trunc_doc=50)
val_dataset = create_graph_dataset(df=df_val, tfidfs_sent=tfidfs_sent, glovemgr=glovemgr, word_blacklist = word_blacklist, doc_column_name="docs", labels_column_name="labels", is_sep_n=False, remove_stop_word = False, stemming=False, trunc_sent=-1, padding_sent=-1, trunc_doc=50)
test_dataset = create_graph_dataset(df=df_test, tfidfs_sent=tfidfs_sent, glovemgr=glovemgr, word_blacklist = word_blacklist, doc_column_name="docs", labels_column_name="labels", is_sep_n=False, remove_stop_word = False, stemming=False, trunc_sent=-1, padding_sent=-1, trunc_doc=50)

In [52]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader =  DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader =  DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model

In [53]:
class HeterSUMGraph(torch.nn.Module):
  def __init__(self):
    super(HeterSUMGraph, self).__init__()
    self.fc = torch.nn.Linear(in_features=1,out_features=1) # delete that
    # TODO

  def forward(self, data, nb_sent):# In the real implémentation remove nb_sent
    # TODO
    return [1 for _ in range(len(nb_sent))]

In [54]:
model = HeterSUMGraph().to(device=device)

In [55]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train

In [56]:
if not os.path.exists(checkpoints_folder):
  os.makedirs(checkpoints_folder)

In [57]:
arr_train_loss = []
arr_val_loss = []

t1 = time()

for epoch in range(1, epochs+1):
    model.train()
    nb_batch_train = 0
    total_train_loss = 0
    
with tqdm(train_loader, unit="batch", total=len(train_loader)) as tepoch:
    for batch in tepoch:
        tepoch.set_description(f"Epoch {epoch}")
        # TODO

t2 = time()
print("Training duration =", t2-t1)

  0%|          | 0/18 [00:00<?, ?batch/s]/home/rsaid/anaconda3/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'x', 'edge_index', 'edge_attr', 'undirected'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
Epoch 100: 100%|██████████| 18/18 [00:00<00:00, 188.71batch/s]

Training duration = 0.0984644889831543
